In [2]:
# Import the libraries 

import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

In [4]:
top_dests_ktm=pd.read_csv("../../Datasets/destinations_of_kathmandu_updated_with_latlong.csv")
print(top_dests_ktm.head())

                  title avg_rating  voted_by            genre    latitude  \
0      Boudhanath Stupa        4.5      8897  Religious Sites  27.7215062   
1  Swayambhunath Temple        4.5      6203  Religious Sites  27.7149298   
2  Pashupatinath Temple        4.5      4937  Religious Sites  27.7104461   
3     Chandragiri Hills        4.5       399        Mountains  27.6710496   
4       Kopan Monastery        4.5       787  Religious Sites  27.7425438   

   longitude  
0  85.359809  
1  85.288146  
2  85.346503  
3  85.262664  
4  85.362208  


In [5]:
# select the first 95 destinations as they're the destinations having the latitude and longitude currently
top_dests_ktm=top_dests_ktm[:95]
print(top_dests_ktm.tail())

                         title avg_rating  voted_by  \
90     Ghanta Ghar Clock Tower        3.5        11   
91  Jamchen Lhakhang Monastery        3.5         3   
92                    Big Bell        3.5         9   
93       Mrigasthali Deer Park          4         5   
94            Universal Crafts          5         3   

                              genre    latitude  longitude  
90   Points of Interest & Landmarks   27.707477  85.314711  
91                  Religious Sites  27.7215058  85.359192  
92   Points of Interest & Landmarks  27.7268583  85.300625  
93  Nature & Wildlife Areas • Parks   27.711512  85.349772  
94    Art Galleries • Antique Shops  27.7173331  85.381252  


In [6]:
# since we'll be dealing only with latitude and longitude at the moment
# only filter those columns along with the title
print(top_dests_ktm[['title','latitude','longitude']])

                         title    latitude  longitude
0             Boudhanath Stupa  27.7215062  85.359809
1         Swayambhunath Temple  27.7149298  85.288146
2         Pashupatinath Temple  27.7104461  85.346503
3            Chandragiri Hills  27.6710496  85.262664
4              Kopan Monastery  27.7425438  85.362208
..                         ...         ...        ...
90     Ghanta Ghar Clock Tower   27.707477  85.314711
91  Jamchen Lhakhang Monastery  27.7215058  85.359192
92                    Big Bell  27.7268583  85.300625
93       Mrigasthali Deer Park   27.711512  85.349772
94            Universal Crafts  27.7173331  85.381252

[95 rows x 3 columns]


In [7]:
# let's just pick n=10 for now for better visualization
n=10 #graph size
top_n_dests=top_dests_ktm[:n]
print(top_n_dests)

                     title avg_rating  voted_by  \
0         Boudhanath Stupa        4.5      8897   
1     Swayambhunath Temple        4.5      6203   
2     Pashupatinath Temple        4.5      4937   
3        Chandragiri Hills        4.5       399   
4          Kopan Monastery        4.5       787   
5                   Thamel          4      5687   
6         Garden of Dreams        4.5      4021   
7      Namo Buddha (Stupa)        4.5       129   
8  Kathmandu Durbar Square          4      4711   
9                     Asan        4.5       243   

                                              genre    latitude  longitude  
0                                   Religious Sites  27.7215062  85.359809  
1                                   Religious Sites  27.7149298  85.288146  
2                                   Religious Sites  27.7104461  85.346503  
3                                         Mountains  27.6710496  85.262664  
4                                   Religious Sites  

In [8]:
# also meanwhile let's create a matrix called 'graph' to store the weights for each edges
graph=np.zeros((n,n))

def create_graph():

    # iterators to iterate through the graph
    i=0
    j=0

    for lat,long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
        j=0
        for another_lat,another_long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
            #edge weight(Euclidean distance)
            distance_between_the_places=np.sqrt((lat-another_lat)**2+(long-another_long)**2)
            #print(distance_between_the_places)

            #and store it in the 'graph' matrix
            graph[i][j]=distance_between_the_places
            #increment j 
            j+=1

        #increment i
        i+=1

In [9]:
create_graph()
print(graph)

[[0.         0.07196422 0.01730239 0.10946671 0.021174   0.05232003
  0.04806329 0.2654699  0.05799622 0.05155153]
 [0.07196422 0.         0.05852919 0.05074233 0.07904304 0.01974826
  0.02418316 0.32390089 0.01954512 0.02327766]
 [0.01730239 0.05852919 0.         0.09263384 0.03573404 0.03887774
  0.03438489 0.27060331 0.04250484 0.03642277]
 [0.10946671 0.05074233 0.09263384 0.         0.12255802 0.06307234
  0.06573315 0.3309644  0.05333185 0.05989373]
 [0.021174   0.07904304 0.03573404 0.12255802 0.         0.06089542
  0.05741827 0.27613055 0.06933144 0.06272094]
 [0.05232003 0.01974826 0.03887774 0.06307234 0.06089542 0.
  0.00450891 0.30640256 0.0113667  0.00786942]
 [0.04806329 0.02418316 0.03438489 0.06573315 0.05741827 0.00450891
  0.         0.30209226 0.01267828 0.00696979]
 [0.2654699  0.32390089 0.27060331 0.3309644  0.27613055 0.30640256
  0.30209226 0.         0.30450329 0.30086202]
 [0.05799622 0.01954512 0.04250484 0.05333185 0.06933144 0.0113667
  0.01267828 0.304503

In [10]:
from numpy import argmin

## Now let's define the functions for implementing the 2-opt 

#### In a nutshell, 2-opt improves the path obtained from the greedy approach

In [11]:
def _swap_2opt(route, i, k):
    """ 
        Swapping the route 
    """
    new_route = route[0:i]
    new_route.extend(reversed(route[i:k + 1]))
    new_route.extend(route[k + 1:])
    return new_route


In [12]:
def route_cost(graph, path):
    """
        Calculates the cost for the route. 
    """
    cost = 0
    for index in range(len(path) - 1):
        cost = cost + graph[path[index]][path[index + 1]]
    # add last edge to form a cycle.
    cost = cost + graph[path[-1], path[0]]
    return cost

In [13]:
# implement the 2-opt heuristic to improve the returned greedy route

def tsp_2_opt(graph, route):
    """
    Approximate the optimal path of travelling salesman according to 2-opt algorithm
    Args:
        graph: 2d numpy array as graph
        route: list of nodes

    Returns:
        optimal path according to 2-opt algorithm

    Examples:
        >>> import numpy as np
        >>> graph = np.array([[  0, 300, 250, 190, 230],
        >>>                   [300,   0, 230, 330, 150],
        >>>                   [250, 230,   0, 240, 120],
        >>>                   [190, 330, 240,   0, 220],
        >>>                   [230, 150, 120, 220,   0]])
        >>> tsp_2_opt(graph)
    """
    improved = True
    best_found_route = route
    best_found_route_cost = route_cost(graph, best_found_route)
    while improved:
        improved = False
        for i in range(1, len(best_found_route) - 1):
            for k in range(i + 1, len(best_found_route) - 1):
                new_route = _swap_2opt(best_found_route, i, k)
                new_route_cost = route_cost(graph, new_route)
                if new_route_cost < best_found_route_cost:
                    best_found_route_cost = new_route_cost
                    best_found_route = new_route
                    improved = True
                    break
            if improved:
                break
    return best_found_route_cost,best_found_route


### The following is the implementation of the greedy approach i.e. using the Nearest Neighbor Heuristic

In [14]:
data= list(range(0,n))# should contain the list of city ids 
matrix=graph

In [15]:
# redefining the nearest neighbour heuristic

def nearest_neighbor_heuristic_route_generator(starting_point):
    """
        Greedy Nearest neighbor heuristic route generator
    """
    # maintain a list called 'visited' to store the cities that have been visited
    # initially, this holds the starting city
    visited=[]
    visited.append(starting_point)
    current_point=starting_point
    #also maintain a local copy of all the cities
    data_copy=data.copy()
    #remove the visited city from this list
    data_copy.remove(starting_point)
    #total cost of the route
    total_cost=0
    
    # from the starting city, calculate the cost to every cities(except itself) and select the city with minimum cost
    while data_copy !=[]:
        #calculate cost to every other cities in the cities_copy from the current city and store in cost_list
        cost_list=[]
        for neighbor_point in data_copy:
            cost=matrix[current_point][neighbor_point]
            cost_list.append(cost)
        print(cost_list)
        min_idx=np.argmin(cost_list)#selects the index with the minimum cost
        print(min_idx) 
        #also add the min cost to the total cost
        total_cost+=min(cost_list)
        #now find the corresponding point (neighbor)
        corresponding_point=data_copy[min_idx]
        # add that to the visited list
        visited.append(corresponding_point)
        # remove that from the data_copy list
        data_copy.remove(corresponding_point)
        # change current city to the city with min cost (corresponding city)
        current_point=corresponding_point
        
    # last city has to be connected to the starting city so let's do that
    total_cost+=matrix[current_point][starting_point] #add to total cost
    visited.append(starting_point)
    
    return total_cost,visited
        

#display the result for the data
total_cost,route_generated=nearest_neighbor_heuristic_route_generator(0)
        
print("Total cost:",total_cost)
print("Route generated:",route_generated)
    

[0.07196421984966779, 0.017302392517227098, 0.10946671284756053, 0.021173998536175558, 0.05232003469762648, 0.04806328558161538, 0.2654699039776977, 0.057996219854138865, 0.05155153188121596]
1
[0.058529192353306815, 0.09263384153585069, 0.03573404447372474, 0.03887774439264948, 0.034384890849900394, 0.27060331294677503, 0.042504837102725034, 0.03642277351163117]
4
[0.02418315872111161, 0.06573314517205395, 0.0574182650828934, 0.004508906423965498, 0.3020922644719783, 0.012678282680630197, 0.006969785970889896]
3
[0.0197482572155125, 0.06307233500584433, 0.060895419772675555, 0.3064025581614035, 0.011366697860413052, 0.007869424966288062]
5
[0.023277662275460145, 0.059893734444014586, 0.06272094488262701, 0.30086201737349305, 0.0066249377815913195]
4
[0.019545119288466608, 0.053331854393040265, 0.06933144462045371, 0.3045032885189848]
0
[0.05074232839158659, 0.07904303710486275, 0.32390088660650934]
0
[0.12255802049347642, 0.33096439502236213]
0
[0.2761305485408664]
0
Total cost: 0.805

In [16]:
print("Total cost (greedy):",total_cost)
print("Route generated:",route_generated)

Total cost (greedy): 0.8051364732310662
Route generated: [0, 2, 6, 5, 9, 8, 1, 3, 4, 7, 0]


In [17]:
for i in route_generated:
    print(top_n_dests.iloc[i]["title"],end='')
    print("--->",end='')

Boudhanath Stupa--->Pashupatinath Temple--->Garden of Dreams--->Thamel--->Asan--->Kathmandu Durbar Square--->Swayambhunath Temple--->Chandragiri Hills--->Kopan Monastery--->Namo Buddha (Stupa)--->Boudhanath Stupa--->

In [18]:
#apply 2-opt 
final_cost,final_route=tsp_2_opt(np.array(matrix),route_generated)
print("Final cost:",final_cost)
print("Final route:",final_route)

Final cost: 0.786044538260869
Final route: [0, 7, 2, 9, 8, 3, 1, 5, 6, 4, 0]


In [19]:
for i in final_route:
    print(top_n_dests.iloc[i]["title"],end='')
    print("--->",end='')

Boudhanath Stupa--->Namo Buddha (Stupa)--->Pashupatinath Temple--->Asan--->Kathmandu Durbar Square--->Chandragiri Hills--->Swayambhunath Temple--->Thamel--->Garden of Dreams--->Kopan Monastery--->Boudhanath Stupa--->

In [23]:
#actual distance given by Vincenty distance using more accurate ellipsoidal models such as WGS-84 than Haversine
import geopy.distance
total_distance=0 #total actual distance

for i in range(len(final_route)-1):
    coords_1 = (top_n_dests.iloc[final_route[i]]["latitude"],top_n_dests.iloc[final_route[i]]["longitude"])
    coords_2 = (top_n_dests.iloc[final_route[i+1]]["latitude"],top_n_dests.iloc[final_route[i+1]]["longitude"])
    
    #names of destinations connected
    src=top_n_dests.iloc[final_route[i]]["title"]
    dest=top_n_dests.iloc[final_route[i+1]]["title"]
    distance=geopy.distance.geodesic(coords_1, coords_2).km
    print (f'{str(src)+"->"+str(dest)}',distance)
    total_distance=total_distance+distance

print("Total distance(km): ",total_distance)

Boudhanath Stupa->Namo Buddha (Stupa) 27.28683809434689
Namo Buddha (Stupa)->Pashupatinath Temple 27.62640690201228
Pashupatinath Temple->Asan 3.595305941383428
Asan->Kathmandu Durbar Square 0.6745014662557417
Kathmandu Durbar Square->Chandragiri Hills 5.520273939092278
Chandragiri Hills->Swayambhunath Temple 5.473767406580083
Swayambhunath Temple->Thamel 1.947566705973273
Thamel->Garden of Dreams 0.44689862662388247
Garden of Dreams->Kopan Monastery 5.8413472683014165
Kopan Monastery->Boudhanath Stupa 2.343260122303754
Total distance(km):  80.75616647287303
